# Capstone project - analyzing canadian boroughs.

All the code is in this one file. The two subsections tell about the three subproblems.


## Scaping data from Wikipedia

In [166]:
# Importing minimun libraries
import numpy as np
import pandas as pd

In [167]:
# Get the data set from wikipedia
 
postal_codes_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]


# Look at the data
postal_codes_df.head()


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [168]:
# Removing not assigned neighborhoods and checking the outcome
borough_prune_df = postal_codes_df.loc[postal_codes_df.Borough != 'Not assigned']
borough_prune_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [169]:
# We look at the shape of the result
postal_codes_df.shape

(180, 3)

In [170]:
# Then we check if the shape changes after removing empty elements. It does not, so we are happy.
postal_codes_df.dropna
postal_codes_df.shape

(180, 3)

In [171]:
postal_codes_df.shape

(180, 3)

## Getting the latitudes and longitudes

We could not import geocoder, so we are using the provided csv file.


In [172]:
# Reading the csv from the given site
langlot_df = pd.read_csv("https://cocl.us/Geospatial_data")

In [173]:
# Checking the data
langlot_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [174]:
# We turn the postal code label to a more accessible form and set it as the index for joining
left_df = postal_codes_df.rename(columns={'Postal Code':'Postal_Code'})
right_df = langlot_df.rename(columns={'Postal Code':'Postal_Code'})

left_df.set_index('Postal_Code', inplace=True)
right_df.set_index('Postal_Code', inplace=True)

In [175]:
# we join the data sets along the index

combined_df = left_df.join(right_df)
combined_df = combined_df.loc[combined_df.Borough != 'Not assigned']

combined_df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal_Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [176]:
combined_df.shape

(103, 4)

## Studying the data


Finally we do a few visualizations of the data. We plot the neighbourhoods and cluster them into three parts based on 


In [177]:
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

In [178]:
CAN_lat = 43.6532
CAN_lon = -79.3832

In [179]:
# Creating a map instance
map_CAN = folium.Map(location=[CAN_lat, CAN_lon], zoom_start=10)

# Adding markers to map (Using the code from the lab)
for lat, lng, borough, neighborhood in zip(combined_df['Latitude'], combined_df['Longitude'], combined_df['Borough'], combined_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CAN)  
    
map_CAN

In [180]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [181]:
# set number of clusters
kclusters = 3

CAN_grouped_clustering = combined_df.drop(['Neighborhood', 'Borough'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CAN_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([1, 1, 0, 2, 0, 2, 1, 0, 1, 0], dtype=int32)

In [182]:
CAN_merged = combined_df.copy()

CAN_merged.insert(0, 'Cluster Labels', kmeans.labels_)

Finally we visualize the clusters

In [184]:
#kcclusters

# CAN_merged


# create map
map_clusters = folium.Map(location=[CAN_lat, CAN_lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CAN_merged['Latitude'], CAN_merged['Longitude'], CAN_merged['Neighborhood'], CAN_merged['Cluster Labels']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters